# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [2]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('../data/kc_house_data.gl/')
print sales.shape

This non-commercial license of GraphLab Create for academic use is assigned to bmatthewtaylor@gmail.com and will expire on October 27, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\username\AppData\Local\Temp\graphlab_server_1483582806.log.0


(21613, 21)


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [5]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
#create additional column sqrt('sqft_living')
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
#create additional column sqrt('sqft_lot')
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
#create additional column 'bedrooms'^2

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
# convert column from string to float
sales['floors_square'] = sales['floors']*sales['floors']
#create new column 'floors'^2
print sales.column_names()
print sales.shape  #nb: additional 4 columns

['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'sqft_living_sqrt', 'sqft_lot_sqrt', 'bedrooms_square', 'floors_square']
(21613, 25)


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [7]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']
print len(all_features)
#NB above list excludes these columns  ['zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']

17


In [8]:
model_all = graphlab.linear_regression.create(sales, 
                                              target='price', 
                                              features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., 
                                              l1_penalty=1e10, 
                                              verbose=False)
print "sales.shape = ", sales.shape
print "len(all_features) = ", len(all_features)
print "model_all.coefficients.shape = ", model_all.coefficients.shape
#NB: one more coefficient than in list of features because first element is intercept

sales.shape =  (21613, 25)
len(all_features) =  17
model_all.coefficients.shape =  (18, 4)


Find what features had non-zero weight.

In [108]:
#print "type(model_all) = ", type(model_all)
print "model_all.coefficients.column_names() = ", model_all.coefficients.column_names()
model_all_coeffs = model_all.coefficients['value']

print "len(model_all.coefficients['value']) = ", len(model_all.coefficients['value'])
#print type(model_all_coeffs)
#<class 'graphlab.data_structures.sarray.SArray'>
print "model_all.coefficients['value'] = ", model_all.coefficients['value']
print "model_all_coeffs.non zero coefficients = ", model_all.coefficients['value'].nnz(), \
    ", total coefficients = ", len(model_all.coefficients['value'])

print "\nnon zero coefficients, names and values"
for i in range(len(model_all.coefficients['value'])):
    if model_all.coefficients['value'][i]!=0:
        print model_all.coefficients['name'][i], " = ", model_all.coefficients['value'][i]
print "\nzero value coefficients"
for i in range(len(model_all.coefficients['value'])):
    if model_all.coefficients['value'][i]==0:
        print model_all.coefficients['name'][i]
print "\n______________________"
print model_all.coefficients.print_rows(num_rows=18)

model_all.coefficients.column_names() =  ['name', 'index', 'value', 'stderr']
len(model_all.coefficients['value']) =  18
model_all.coefficients['value'] =  [274873.0559504957, 0.0, 0.0, 8468.531086910072, 24.420720982445214, 350.0605533860648, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 842.0680348976282, 20.024722417091304, 0.0, 0.0, 0.0]
model_all_coeffs.non zero coefficients =  6 , total coefficients =  18

non zero coefficients, names and values
(intercept)  =  274873.05595
bathrooms  =  8468.53108691
sqft_living  =  24.4207209824
sqft_living_sqrt  =  350.060553386
grade  =  842.068034898
sqft_above  =  20.0247224171

zero value coefficients
bedrooms
bedrooms_square
sqft_lot
sqft_lot_sqrt
floors
floors_square
waterfront
view
condition
sqft_basement
yr_built
yr_renovated

______________________
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.0

In [33]:
#replicate above as a function
def list_non_zero_coeffs(model):
    print "model.coefficients.shape = ", model.coefficients.shape
    model_all_coeffs = model.coefficients['value']
    non_zero_coeffs_list = []
    for i in range(len(model_all_coeffs)-1):
        if model_all_coeffs[i]!=0:
            print model.coefficients['name'][i], " \t= \t", model_all_coeffs[i]
            non_zero_coeffs_list.append(model.coefficients['name'][i])
    print "\nzero value coefficients"
    for i in range(len(model_all_coeffs)-1):
        if model_all_coeffs[i]==0:
            print all_features[i]
    return non_zero_coeffs_list
nonzero_list = list_non_zero_coeffs(model_all)
#NB: intercept is included in this list. intercept <> weight on feature
print "# of non zero features = ", len(nonzero_list)

model.coefficients.shape =  (18, 4)
(intercept)  	= 	274873.05595
bathrooms  	= 	8468.53108691
sqft_living  	= 	24.4207209824
sqft_living_sqrt  	= 	350.060553386
grade  	= 	842.068034898
sqft_above  	= 	20.0247224171

zero value coefficients
bedrooms_square
bathrooms
sqft_lot_sqrt
floors
floors_square
waterfront
view
condition
grade
yr_built
yr_renovated
# of non zero features =  6


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [34]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

In [38]:
print "training_and_validation.shape = ", training_and_validation.shape
print "testing = ", testing.shape
print "training.shape = ", training.shape
print "validation.shape = ", validation.shape
col_names = training.column_names()
print "training.column_names() = ", col_names
#del col_names[2]#delete 'price'
#del col_names[1] #delete 'date'
#print "\ncol_names after deleting date = ", col_names
#training = training[col_names]
#print "\n", training.column_names()

training_and_validation.shape =  (19396, 25)
testing =  (2217, 25)
training.shape =  (9761, 24)
validation.shape =  (9635, 25)
training.column_names() =  ['id', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'sqft_living_sqrt', 'sqft_lot_sqrt', 'bedrooms_square', 'floors_square']


Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [39]:
l1_penalty_series = np.logspace(1, 7, num=13)
print "l1_penalty_series = ", l1_penalty_series
print "len(l1_penalty_series) = ", len(l1_penalty_series)

l1_penalty_series =  [  1.00000000e+01   3.16227766e+01   1.00000000e+02   3.16227766e+02
   1.00000000e+03   3.16227766e+03   1.00000000e+04   3.16227766e+04
   1.00000000e+05   3.16227766e+05   1.00000000e+06   3.16227766e+06
   1.00000000e+07]
len(l1_penalty_series) =  13


In [40]:
rss_stored = []
i = 0
for l1_penalty in l1_penalty_series:
    #https://turi.com/products/create/docs/generated/graphlab.linear_regression.create.html
    model = graphlab.linear_regression.create(training, 
                                              target='price', 
                                              features= all_features,
                                              #NB: previously we excluded some columns.
                                              #this causes errors if we exclude 'price' from features list.
                                              l2_penalty=0., 
                                              l1_penalty=l1_penalty, 
                                              validation_set=None, 
                                              verbose=False)
    
    #print type(model) 
    #graphlab.toolkits.regression.linear_regression.LinearRegression
    print "model.coefficients.shape = ", model.coefficients.shape
    predicted = model.predict(validation)
    #NB: user model trained on training, to predict values in validation set.
    error = predicted - validation['price']
    rss = (error**2).sum()
    #rss = sum squared errors
    print "i=", i, ", l1_penalty = ", l1_penalty, ", rss = ", rss
    rss_stored.append(rss)
    i += 1

model.coefficients.shape =  (18, 4)
i= 0 , l1_penalty =  10.0 , rss =  6.25766285142e+14
model.coefficients.shape =  (18, 4)
i= 1 , l1_penalty =  31.6227766017 , rss =  6.25766285362e+14
model.coefficients.shape =  (18, 4)
i= 2 , l1_penalty =  100.0 , rss =  6.25766286058e+14
model.coefficients.shape =  (18, 4)
i= 3 , l1_penalty =  316.227766017 , rss =  6.25766288257e+14
model.coefficients.shape =  (18, 4)
i= 4 , l1_penalty =  1000.0 , rss =  6.25766295212e+14
model.coefficients.shape =  (18, 4)
i= 5 , l1_penalty =  3162.27766017 , rss =  6.25766317206e+14
model.coefficients.shape =  (18, 4)
i= 6 , l1_penalty =  10000.0 , rss =  6.25766386761e+14
model.coefficients.shape =  (18, 4)
i= 7 , l1_penalty =  31622.7766017 , rss =  6.25766606749e+14
model.coefficients.shape =  (18, 4)
i= 8 , l1_penalty =  100000.0 , rss =  6.25767302792e+14
model.coefficients.shape =  (18, 4)
i= 9 , l1_penalty =  316227.766017 , rss =  6.25769507644e+14
model.coefficients.shape =  (18, 4)
i= 10 , l1_penalty 

In [41]:
print "min rss = ", min(rss_stored)
print "index of min rss = ", rss_stored.index(min(rss_stored))
l1_penalty_minRSS = l1_penalty_series[rss_stored.index(min(rss_stored))]
print "l1_penalty = ", l1_penalty_minRSS

min rss =  6.25766285142e+14
index of min rss =  0
l1_penalty =  10.0


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [49]:
model = graphlab.linear_regression.create(training, 
                                          target='price', 
                                          features=all_features,
                                          l2_penalty=0., 
                                          l1_penalty=l1_penalty_minRSS, 
                                          validation_set=None, 
                                          verbose=False)
print "model.coefficients.shape = ", model.coefficients.shape
#NB: need to set features = list, o/wise function builds non sensible model.

#NBB: check if we are supposed to calculated this on testing or validation
predicted = model.predict(testing)
error = predicted - testing['price']
rss = (error**2).sum()
#rss = sum squared errors
print "rss = ", rss
print model.coefficients.print_rows(num_rows=18)
print model.coefficients['value'].nnz()

model.coefficients.shape =  (18, 4)
rss =  1.56983602382e+14
+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None 

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [ ]:
#print list_non_zero_coeffs(model)
print training.shape
print model.coefficients.shape

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [55]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [50]:
l1_penalty_values = np.logspace(8, 10, num=20)
print "l1_penalty_values = ", l1_penalty_values

l1_penalty_values =  [  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [52]:
model_coefficients_values_nnz = []
rss_stored = []
i = 0
for l1_penalty in l1_penalty_values:
    #https://turi.com/products/create/docs/generated/graphlab.linear_regression.create.html
    model = graphlab.linear_regression.create(training, 
                                              target='price', 
                                              features= all_features,
                                              #NB: previously we excluded some columns.
                                              #this causes errors if we exclude 'price' from features list.
                                              l2_penalty=0., 
                                              l1_penalty=l1_penalty, 
                                              validation_set=None, 
                                              verbose=False)
    print "model.coefficients.shape = ", model.coefficients.shape
    predicted = model.predict(validation)
    #NB: user model trained on training, to predict values in validation set.
    error = predicted - validation['price']
    rss = (error**2).sum()
    #rss = sum squared errors
    model_coefficients_values_nnz.append(model.coefficients['value'].nnz())
    print "i=", i, ", l1_penalty = ", l1_penalty, ", rss = ", rss, ", non zero weights = ", model_coefficients_values_nnz[i]
    rss_stored.append(rss)
    i += 1

model.coefficients.shape =  (18, 4)
i= 0 , l1_penalty =  100000000.0 , rss =  6.27492659875e+14 , non zero weights =  18
model.coefficients.shape =  (18, 4)
i= 1 , l1_penalty =  127427498.57 , rss =  6.28210516771e+14 , non zero weights =  18
model.coefficients.shape =  (18, 4)
i= 2 , l1_penalty =  162377673.919 , rss =  6.29176689541e+14 , non zero weights =  18
model.coefficients.shape =  (18, 4)
i= 3 , l1_penalty =  206913808.111 , rss =  6.30650082719e+14 , non zero weights =  18
model.coefficients.shape =  (18, 4)
i= 4 , l1_penalty =  263665089.873 , rss =  6.32940229287e+14 , non zero weights =  17
model.coefficients.shape =  (18, 4)
i= 5 , l1_penalty =  335981828.628 , rss =  6.3626814023e+14 , non zero weights =  17
model.coefficients.shape =  (18, 4)
i= 6 , l1_penalty =  428133239.872 , rss =  6.41261198311e+14 , non zero weights =  17
model.coefficients.shape =  (18, 4)
i= 7 , l1_penalty =  545559478.117 , rss =  6.48983455376e+14 , non zero weights =  17
model.coefficients.s

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [65]:
#recall max_nonzeros = 7
#display values for readability purposes.
print "model_coefficients_values_nnz = ", model_coefficients_values_nnz
print "l1_penalty_values = ", l1_penalty_values

#NB: l1_penalty_values increments in value with index.
#NBB: model_coefficients_values_nnz decreases in value with index.
# thus once we find threshhold point we know both l1_penalty_max and l1_penalty_min
# NBB: logic suits this sequence, better logic needed for general case.
i = 0
l1_penalty_min = -1.
l1_penalty_max = -1.

for l1_penalty in l1_penalty_values:
    #print "i=", i, "l1_penalty=", l1_penalty, "model_coefficients_values_nnz[i]=", model_coefficients_values_nnz[i]
    if model_coefficients_values_nnz[i] < max_nonzeros and l1_penalty_min == -1.:
        l1_penalty_min = l1_penalty_values[i-1]
        l1_penalty_max = l1_penalty_values[i]
        break
    i += 1
print "l1_penalty_min = ", l1_penalty_min
print "l1_penalty_max = ", l1_penalty_max

model_coefficients_values_nnz =  [18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]
l1_penalty_values =  [  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]
l1_penalty_min =  2976351441.63
l1_penalty_max =  3792690190.73


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [67]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
#https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.linspace.html
#numpy.linspace(start, stop, num=50, endpoint=True, retstep=False, dtype=None)[source]
#Return evenly spaced numbers over a specified interval.
print len(l1_penalty_values)
print "l1_penalty_values = ", l1_penalty_values

20
l1_penalty_values =  [  2.97635144e+09   3.01931664e+09   3.06228184e+09   3.10524703e+09
   3.14821223e+09   3.19117743e+09   3.23414263e+09   3.27710782e+09
   3.32007302e+09   3.36303822e+09   3.40600341e+09   3.44896861e+09
   3.49193381e+09   3.53489901e+09   3.57786420e+09   3.62082940e+09
   3.66379460e+09   3.70675980e+09   3.74972499e+09   3.79269019e+09]


* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [68]:
model_coefficients_values_nnz = []
rss_stored = []
i = 0

for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, 
                                              target='price', 
                                              features= all_features,
                                              #NB: previously we excluded some columns.
                                              #this causes errors if we exclude 'price' from features list.
                                              l2_penalty=0., 
                                              l1_penalty=l1_penalty, 
                                              validation_set=None, 
                                              verbose=False)
    #Measure the RSS of the learned model on the VALIDATION set
    print "model.coefficients.shape = ", model.coefficients.shape
    predicted = model.predict(validation)
    #NB: user model trained on training, need to predict values in validation set.
    error = predicted - validation['price']
    rss = (error**2).sum()
    #rss = sum squared errors
    rss_stored.append(rss)
    model_coefficients_values_nnz.append(model.coefficients['value'].nnz())
    print "i=", i, "l1_penalty=", l1_penalty, "model_coefficients_values_nnz[i]=", model_coefficients_values_nnz[i]
    i += 1
#Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to max_nonzeros.


model.coefficients.shape =  (18, 4)
i= 0 l1_penalty= 2976351441.63 model_coefficients_values_nnz[i]= 10
model.coefficients.shape =  (18, 4)
i= 1 l1_penalty= 3019316638.95 model_coefficients_values_nnz[i]= 10
model.coefficients.shape =  (18, 4)
i= 2 l1_penalty= 3062281836.27 model_coefficients_values_nnz[i]= 10
model.coefficients.shape =  (18, 4)
i= 3 l1_penalty= 3105247033.59 model_coefficients_values_nnz[i]= 10
model.coefficients.shape =  (18, 4)
i= 4 l1_penalty= 3148212230.92 model_coefficients_values_nnz[i]= 10
model.coefficients.shape =  (18, 4)
i= 5 l1_penalty= 3191177428.24 model_coefficients_values_nnz[i]= 10
model.coefficients.shape =  (18, 4)
i= 6 l1_penalty= 3234142625.56 model_coefficients_values_nnz[i]= 10
model.coefficients.shape =  (18, 4)
i= 7 l1_penalty= 3277107822.88 model_coefficients_values_nnz[i]= 10
model.coefficients.shape =  (18, 4)
i= 8 l1_penalty= 3320073020.2 model_coefficients_values_nnz[i]= 8
model.coefficients.shape =  (18, 4)
i= 9 l1_penalty= 3363038217.52

In [100]:
#Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to max_nonzeros.
# we have lists model_coefficients_values_nnz, rss_stored, l1_penalty_values
# recall we have max_nonzeros = 7
# we want l1_penalty providing the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.
indices_Max_nonzeros = [i for i, x in enumerate(model_coefficients_values_nnz) if x == 7]
print "indices_Max_nonzeros = ", indices_Max_nonzeros
print "indices_Max_nonzeros[0] = ", indices_Max_nonzeros[0]
print "len(indices_Max_nonzeros) = ", len(indices_Max_nonzeros)
rss_stored_Max_nonzeros = rss_stored[indices_Max_nonzeros[0]:(indices_Max_nonzeros[0]+len(indices_Max_nonzeros))]
print "rss_stored_Max_nonzeros = ", rss_stored_Max_nonzeros
print "min(rss_stored_Max_nonzeros) = ", min(rss_stored_Max_nonzeros)
print "rss_stored_Max_nonzeros.index(min(rss_stored_Max_nonzeros)) = ", rss_stored_Max_nonzeros.index(min(rss_stored_Max_nonzeros))
min_rss_indice_Max_nonzeros = indices_Max_nonzeros[0] + rss_stored_Max_nonzeros.index(min(rss_stored_Max_nonzeros))
print "min_rss_indice_Max_nonzeros = ", min_rss_indice_Max_nonzeros
print "l1_penalty_values[min_rss_indice_Max_nonzeros] = ", l1_penalty_values[min_rss_indice_Max_nonzeros]

indices_Max_nonzeros =  [11, 12, 13, 14]
indices_Max_nonzeros[0] =  11
len(indices_Max_nonzeros) =  4
rss_stored_Max_nonzeros =  [1046937488751711.1, 1051147625612860.9, 1055992735342999.1, 1060799531763287.8]
min(rss_stored_Max_nonzeros) =  1.04693748875e+15
rss_stored_Max_nonzeros.index(min(rss_stored_Max_nonzeros)) =  0
min_rss_indice_Max_nonzeros =  11
l1_penalty_values[min_rss_indice_Max_nonzeros] =  3448968612.16


In [106]:
#NB: for comparison purposes, find the minimum RSS and the corresponding l1_penalty value
print "min RSS = ", min(rss_stored)
print "index of min RSS = ", rss_stored.index(min(rss_stored))
print "l1_penalty for min RSS = ", l1_penalty_values[rss_stored.index(min(rss_stored))]
#NBB : This is obviously not the lowest RSS with sparsity *equal* to `max_nonzeros`.

min RSS =  9.66925692362e+14
index of min RSS =  0
l1_penalty for min RSS =  2976351441.63


In [107]:
model = graphlab.linear_regression.create(training, 
                                          target='price', 
                                          features= all_features, 
                                          l2_penalty=0., 
                                          l1_penalty=l1_penalty_values[min_rss_indice_Max_nonzeros], 
                                          validation_set=None, 
                                          verbose=False)
model_coeffs = model.get("coefficients")
print "model_coeffs.shape = ", model_coeffs.shape
print "model_coeffs.shape[0] = ", model_coeffs.shape[0]
print type(model_coeffs.shape[0])
print type(model_all_coeffs)
print model.get("coefficients")['value']
print model_coeffs.print_rows(num_rows=18)

print "non zero value coefficients"
for i in range(model_coeffs.shape[0]):
    if model.get("coefficients")['value'][i]!=0:
        print "%-*s  = %s" % (20,model.get("coefficients")['name'][i], model.get("coefficients")['value'][i])
print "\nzero value coefficients"
for i in range(model_coeffs.shape[0]):
    if model.get("coefficients")['value'][i]==0:
        print model.get("coefficients")['name'][i]


model_coeffs.shape =  (18, 4)
model_coeffs.shape[0] =  18
<type 'int'>
<class 'graphlab.data_structures.sarray.SArray'>
[222253.19254432785, 661.7227177822587, 0.0, 15873.957259267981, 32.41022145125964, 690.1147733133256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2899.4202697498786, 30.011575302201045, 0.0, 0.0, 0.0]
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0  

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?